## CONVERT RAW DATA TO CSV FORMAT

In [3]:
import pandas as pd
import re

In [4]:
WORD_IDX = 0
SEP = '|'
TAG_IDX = 1
def _read_data(path_to_data='.'):
    data = []
    with open(path_to_data, "r", encoding="utf8") as input_file:
        curr_words = []
        curr_tags = []
        for i, line in enumerate(input_file):
            
            line = line.rstrip()
            if len(line) == 0:
                data.append((curr_words, curr_tags))
                curr_words = []
                curr_tags = []
            else:
                entry = tuple(line.split(SEP))
                try:
                    word = entry[WORD_IDX]
                    tag = entry[TAG_IDX]
                    curr_words.append(word)
                    curr_tags.append(tag)
                except:
                    print('line ', i, ' : ', line)
                    print('entry: ', entry) 
    return data

In [5]:
data = _read_data(r'D:\Google Drive\Disertatie_V2\datasets_vanilla\monero\train')
data = data + _read_data(r'D:\Google Drive\Disertatie_V2\datasets_vanilla\monero\test')
data = data + _read_data(r'D:\Google Drive\Disertatie_V2\datasets_vanilla\monero\dev')

In [6]:
import numpy as np

In [12]:
from copy import deepcopy

In [13]:
idx = 220
data[idx][1])
pred = deepcopy(gt)
gt[9] = 'O'
print(pred)

sent = ' '.join(data[idx][0])
gt =   ' '.join(

TypeError: 'str' object does not support item assignment

In [23]:
tags = list(map(lambda x: x[1], data))
tags_flat = []
for t in tags:
    tags_flat.extend(t)

df = pd.DataFrame()
df['tags'] = tags_flat
df = df.filter(df)
df['tags'] = df['tags'].apply(lambda x: x[2:])
df['tags'].replace('', np.nan, inplace=True)
df = df.dropna()
df['tags'].value_counts(normalize=True)

LEGAL    0.452773
ORG      0.357145
TIME     0.093922
LOC      0.057663
PER      0.038497
Name: tags, dtype: float64

In [ ]:
def convert_to_csv(f_path):
    csv = f'sent_id{SEP}word{SEP}tag\n'
    sent_cnt = 0
    with open(f_path, 'r', encoding='utf8') as f:
        for i, line in enumerate(f):
            if len(line) == 0:
                print(line, i, '\n')
                return
            if i % 10_000 == 0:
                print(i)
            if line[0] in ('#'):
                continue
            elif line[0] == '\n':
                sent_cnt += 1
            else:
                line = line.replace('\t', '\s').strip()
                line = line.split('\s')
                csv = csv + str(sent_cnt) + SEP + f'{line[WORD_POS]}' + SEP + f'{line[TAG_POS]}' + '\n'
    with open(f_path + '.proc', 'w', encoding='utf8') as f:
        f.write(csv)


In [ ]:
convert_to_csv(r'D:\Facultate\Master\Disertatie_V2\ronec_iob.conllup')

## CONVERT DATA TO MRC FORMAT

In [ ]:
import pandas as pd
import tensorflow as tf
import json
import unidecode



In [ ]:
df = pd.read_csv(r'D:\Facultate\Master\Disertatie_V2\RONEC', sep='|', encoding='utf8')
df['word'] = df['word'].apply(lambda x: x.lower())
df['word'] = df['word'].apply(lambda x: unidecode.unidecode(x))

In [ ]:
df_mrc_raw = pd.DataFrame()
df_mrc_raw['words'] = df.groupby('sent_id')['word'].apply(lambda x: list(x))
df_mrc_raw['tags'] = df.groupby('sent_id')['tag'].apply(lambda x: list(x))
df_mrc_raw['input'] = df_mrc_raw.apply(lambda x: (x[0], x[1]), axis=1)

In [ ]:
query_map = {k : k for k in df.tag.apply(lambda x: x[2:]).value_counts().index }
del query_map['']

In [ ]:
def get_span_labels(sentence_tags, inv_label_mapping=None):
    """
    Desc:
        get from token_level labels to list of entities, 
        it doesnot matter tagging scheme is BMES or BIO or BIOUS
    Returns:
        a list of entities
        [(start, end, labels), (start, end, labels)]
    """

    if inv_label_mapping:
        sentence_tags = [inv_label_mapping[i] for i in sentence_tags]

    span_labels = []
    last = "O"
    start = -1 
    for i, tag in enumerate(sentence_tags):
        pos, _ = (None, "O") if tag == "O" else tag.split("-")
        if (pos == "S" or pos == "B" or tag == "O") and last != "O":
            span_labels.append((start, i - 1, last.split("-")[-1]))
        if pos == "B" or pos == "S" or last == "O":
            start = i 
        last = tag 

    if sentence_tags[-1] != "O":
        span_labels.append((start, len(sentence_tags) -1 , sentence_tags[-1].split("-"[-1])))

    return span_labels 

def transform_examples_to_qa_features(query_map, entity_labels, data_instances):
    """
    Desc:
        convert_examples to qa features
    Args:
        query_map: {entity label: entity query}; 
        data_instance 
    """ 
    mrc_ner_dataset = []
    tmp_qas_id = 0 
    for idx, (word_lst, label_lst) in enumerate(data_instances):
        candidate_span_label = get_span_labels(label_lst)
        tmp_query_id = 0 
        for label_idx, tmp_label in enumerate(entity_labels):
            tmp_query_id += 1
            tmp_query = query_map[tmp_label]
            tmp_context = " ".join(word_lst)

            tmp_start_pos = []
            tmp_end_pos = []
            tmp_entity_pos = []

            start_end_label = [(start, end) for start, end, label_content in candidate_span_label if label_content == tmp_label]

            if len(start_end_label) != 0:
                for span_item in start_end_label:
                    start_idx, end_idx = span_item 
                    tmp_start_pos.append(start_idx)
                    tmp_end_pos.append(end_idx)
                    tmp_entity_pos.append("{};{}".format(str(start_idx), str(end_idx)))
                tmp_impossible = False 
            else:
                tmp_impossible = True 
            
            mrc_ner_dataset.append({
                "qas_id": "{}.{}".format(str(tmp_qas_id), str(tmp_query_id)),
                "query": tmp_query,
                "context": tmp_context,
                "entity_label": tmp_label,
                "start_position": tmp_start_pos, 
                "end_position": tmp_end_pos,
                "span_position": tmp_entity_pos, 
                "impossible": tmp_impossible
                })
        tmp_qas_id += 1 

    return mrc_ner_dataset

In [ ]:
querys = transform_examples_to_qa_features(query_map, list(set(query_map.keys())), df_mrc_raw.input.to_list())
querys = list(filter(lambda x: not x['impossible'], querys))
with open(r'D:\\Facultate\\Master\\Disertatie_V2\\RONEC_MRC', 'w', encoding='utf8') as f:
    json.dump(querys, f, indent=4)

## LOAD MRC DATA AND TRAIN MODEL

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece
!pip install tensorflow_addons 

     |████████████████████████████████| 1.2 MB 6.1 MB/s 
     |████████████████████████████████| 1.1 MB 5.4 MB/s 


In [1]:
import pandas as pd
import tensorflow as tf
import json
import numpy as np
import sys
from tensorflow import keras

# constants
BASE_PATH="/content/drive/MyDrive/Disertatie_V2/"
BERT_PATH="BERT"

sys.path.append(BASE_PATH)

# local imports
from repo import dataGenerator
from repo import trainer
from repo.bertLayer import BertLayer

MAX_LENGTH=32


In [2]:
generator_train = dataGenerator.DataGenerator(batch_size=16, base_path=BASE_PATH,
                                              path_to_data="genia/mrc-ner.train", max_seq_length=MAX_LENGTH)

generator_val = dataGenerator.DataGenerator(batch_size=16, base_path=BASE_PATH,
                                              path_to_data="genia/mrc-ner.dev", max_seq_length=MAX_LENGTH)

In [13]:
weighted_bce = lambda x, y: tf.nn.weighted_cross_entropy_with_logits(x, y, pos_weight=10)

losses_ = {
    "span_flat" : weighted_bce,
    "start_flat" : weighted_bce,
    "end_flat" : weighted_bce,
}

losses_weights_ = {
    "span_flat" : 0.34,
    "start_flat" : 0.5,
    "end_flat" : 0.5,
}

_model = trainer.build_function(generator_train.max_seq_length, dropout=.25, base_path=BASE_PATH, bert_path=BERT_PATH)

_model.compile(loss=losses_, loss_weights=losses_weights_, optimizer='adam')

_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 32)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 32)]         0                                            
__________________________________________________________________________________________________
bert_layer_1 (BertLayer)        (None, 32, 768)      177853441   input_ids[0][0]                  
                                                                 input_mask[0][0]           

In [14]:
_model.fit(generator_train, validation_data=generator_val, 
           epochs=10, verbose=1)

Epoch 1/10
4274/4274 [==============================] - 821s 190ms/step - loss: 1.0037 - start_flat_loss: 0.7673 - end_flat_loss: 0.7672 - span_flat_loss: 0.6955 - val_loss: 0.9955 - val_start_flat_loss: 0.7592 - val_end_flat_loss: 0.7592 - val_span_flat_loss: 0.6952
Epoch 2/10
4274/4274 [==============================] - 811s 190ms/step - loss: 1.0027 - start_flat_loss: 0.7662 - end_flat_loss: 0.7662 - span_flat_loss: 0.6954 - val_loss: 0.9955 - val_start_flat_loss: 0.7591 - val_end_flat_loss: 0.7591 - val_span_flat_loss: 0.6952
Epoch 3/10
4274/4274 [==============================] - 809s 189ms/step - loss: 1.0027 - start_flat_loss: 0.7662 - end_flat_loss: 0.7662 - span_flat_loss: 0.6954 - val_loss: 0.9955 - val_start_flat_loss: 0.7591 - val_end_flat_loss: 0.7591 - val_span_flat_loss: 0.6952
Epoch 4/10
4274/4274 [==============================] - 809s 189ms/step - loss: 1.0027 - start_flat_loss: 0.7662 - end_flat_loss: 0.7662 - span_flat_loss: 0.6954 - val_loss: 0.9955 - val_start_fla

In [15]:
_model.save(BASE_PATH + "/trained_model.bin")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Disertatie_V2//trained_model.bin/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Disertatie_V2//trained_model.bin/assets


In [17]:
_model = keras.models.load_model(BASE_PATH + "/trained_model.bin", custom_objects={"BertLayer": BertLayer})
print(_model.summary())

ValueError: ignored

In [ ]:
from typing import Tuple, List

class Tag(object):
    def __init__(self, term, tag, begin, end):
        self.term = term
        self.tag = tag
        self.begin = begin
        self.end = end

    def to_tuple(self):
        return tuple([self.term, self.begin, self.end])

    def __str__(self):
        return str({key: value for key, value in self.__dict__.items()})

    def __repr__(self):
        return str({key: value for key, value in self.__dict__.items()})


def bmes_decode(char_label_list: List[Tuple[str, str]]) -> List[Tag]:
    """
    decode inputs to tags
    Args:
        char_label_list: list of tuple (word, bmes-tag)
    Returns:
        tags
    Examples:
        >>> x = [("Hi", "O"), ("Beijing", "S-LOC")]
        >>> bmes_decode(x)
        [{'term': 'Beijing', 'tag': 'LOC', 'begin': 1, 'end': 2}]
    """
    idx = 0
    length = len(char_label_list)
    tags = []
    while idx < length:
        term, label = char_label_list[idx]
        current_label = label[0]

        # correct labels
        if idx + 1 == length and current_label == "B":
            current_label = "S"

        # merge chars
        if current_label == "O":
            idx += 1
            continue
        if current_label == "S":
            tags.append(Tag(term, label[2:], idx, idx + 1))
            idx += 1
            continue
        if current_label == "B":
            end = idx + 1
            while end + 1 < length and char_label_list[end][1][0] == "M":
                end += 1
            if char_label_list[end][1][0] == "E":  # end with E
                entity = "".join(char_label_list[i][0] for i in range(idx, end + 1))
                tags.append(Tag(entity, label[2:], idx, end + 1))
                idx = end + 1
            else:  # end with M/B
                entity = "".join(char_label_list[i][0] for i in range(idx, end))
                tags.append(Tag(entity, label[2:], idx, end))
                idx = end
            continue
        else:
            idx += 1
    return tags

def extract_flat_spans(start_pred, end_pred, pseudo_tag = "TAG"):
    """
    Extract flat-ner spans from start/end/match logits
    Args:
        start_pred: [seq_len], 1/True for start, 0/False for non-start
        end_pred: [seq_len, 2], 1/True for end, 0/False for non-end
        match_pred: [seq_len, seq_len], 1/True for match, 0/False for non-match
        label_mask: [seq_len], 1 for valid boundary.
    Returns:
        tags: list of tuple (start, end)
    Examples:
        >>> start_pred = [0, 1]
        >>> end_pred = [0, 1]
        >>> match_pred = [[0, 0], [0, 1]]
        >>> label_mask = [1, 1]
        >>> extract_flat_spans(start_pred, end_pred, match_pred, label_mask)
        [(1, 2)]
    """
    pseudo_input = "a"

    bmes_labels = ["O"] * len(start_pred)
    start_positions = [idx for idx, tmp in enumerate(start_pred) if tmp]
    end_positions = [idx for idx, tmp in enumerate(end_pred) if tmp]

    for start_item in start_positions:
        bmes_labels[start_item] = f"B-{pseudo_tag}"
    for end_item in end_positions:
        bmes_labels[end_item] = f"E-{pseudo_tag}"

    for tmp_start in start_positions:
        tmp_end = [tmp for tmp in end_positions if tmp >= tmp_start]
        if len(tmp_end) == 0:
            continue
        else:
            tmp_end = min(tmp_end)
            if tmp_start != tmp_end:
                for i in range(tmp_start+1, tmp_end):
                    bmes_labels[i] = f"M-{pseudo_tag}"
            else:
                bmes_labels[tmp_end] = f"S-{pseudo_tag}"
    input_ = [(pseudo_input, label) for label in bmes_labels]
    tags = bmes_decode(input_)
    return [(entity.begin, entity.end, entity.tag) for entity in tags]

In [ ]:
with open(BASE_PATH + "/genia/mrc-ner.dev", "r", encoding="utf8") as f:
    raw_dataset=json.load(f)


## Testing results


In [ ]:
generator_test = dataGenerator.DataGenerator(batch_size=1, base_path=BASE_PATH,
                                              path_to_data="genia/mrc-ner.dev", max_seq_length=MAX_LENGTH)

In [ ]:
def test_model(idx):
  IDX = idx

  raw_example=raw_dataset[IDX]
  example = generator_test[IDX][1]
  pred_example = _model.predict(generator_test[IDX][0])

  start_gt = list(example['start_flat'].flatten())
  end_gt = list(example['end_flat'].flatten())

  gt_entities = extract_flat_spans(start_gt, end_gt, raw_example['entity_label'])

  if len(gt_entities) == 0:
    return

  start_pred = pred_example[0].flatten()
  end_pred = pred_example[1].flatten()

  print(idx)
  print(start_gt)
  print(start_pred)

  start_pred = list(np.round(start_pred))
  end_pred = list(np.round(end_pred))

  pred_entities = extract_flat_spans(start_pred, end_pred, raw_example['entity_label'])
  
  print('gt: ', gt_entities)
  print('pred: ', pred_entities)
  print()

In [ ]:
rand_indexes = np.random.choice(list(range(len(generator_test))), 10)
for idx in rand_indexes:
  test_model(idx)

In [1]:
import pandas as pd

In [10]:
data = []
with open(r'D:\Google Drive\Disertatie_V2\datasets_vanilla\monero_raw\MoNERo.txt', 'r', encoding='utf8') as f:
    sent = []
    tag = []
    for line in f:
        line = line.rstrip()
        if len(line) == 0:
            assert len(sent) == len(tag)
            data.append((sent, tag))

            sent = []
            tag = []
            continue
        toks = line.split()
        sent.append(toks[0])
        tag.append(toks[-2])


In [12]:
import random

random.shuffle(data)

size_ = len(data)

data_train = data[:int(.7*size_)]
data_val = data[int(.7*size_)+1:int(.8*size_)]
data_test = data[int(.8*size_)+1:]

In [13]:
f_names = ("train", "val", "test")
datas = (data_train, data_val, data_test)

for f_name, data in zip(f_names, datas):
    with open(f"D:\\Google Drive\\Disertatie_V2\\datasets_vanilla\\monero_raw\\{f_name}", "w", encoding="utf8") as f:
        for data_point in data:
            sent = data_point[0]
            tag = data_point[1]

            for i in range(len(sent)):
                


[(['Nu', 'am', 'avut', 'complicații', 'intra', 'sau', 'postoperatorii', '.'],
  ['O', 'O', 'O', 'B-DISO', 'I-DISO', 'O', 'I-DISO', 'O']),
 (['Incidența',
   'hemoragiilor',
   'majore',
   'și',
   'a',
   'AVC',
   'hemoragic',
   'nu',
   'a',
   'înregistrat',
   'o',
   'diferență',
   'semnificativă',
   'statistic',
   'la',
   'pacienții',
   'cu',
   'vârste',
   'mai',
   'mari',
   'de',
   '75',
   'de',
   'ani',
   '(',
   '20,29',
   ')',
   '.'],
  ['O',
   'B-DISO',
   'O',
   'O',
   'O',
   'B-DISO',
   'I-DISO',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['Proteazele',
   'din',
   'familia',
   'calpainelor',
   'au',
   'fost',
   'implicate',
   ',',
   'printre',
   'altele',
   ',',
   'în',
   'reglarea',
   'diferențierii',
   'adipocitelor',
   ',',
   'în',
   'procesul',
   'de',
   'down-regulation',
   'al',
   'IRS',
   'mediat